In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

if torch.cuda.is_available(): DEVICE = torch.device('cuda')
else: DEVICE = torch.device('cpu')
OUTPUT_DIM = 2
print(DEVICE)

cpu


In [36]:
try:
    all_x = pd.read_csv("x_train.csv", dtype={"ID": "int"})
    X_test = pd.read_csv("x_test.csv", dtype={"ID": "int"})
except:
    print("Please first run 1_preprocess.ipynb!")
all_x.rename(columns={"Unnamed: 0": "Hour"}, inplace=True)
X_test.rename(columns={"Unnamed: 0": "Hour"}, inplace=True)
all_y = pd.read_csv("train_outcome.csv")
result_df = pd.read_csv("test_nolabel.csv")
pred_proba_df = pd.DataFrame()

# Remember the sequence length (number of rows) for each patient.
# It can help transform the 2D dataframe into 3D tensor later.
max_hour = np.array((), dtype="int16")
for cur_id in all_y["ID"]:
    max_hour = np.append(arr=max_hour, values=np.max(all_x[all_x["ID"] == cur_id]["Hour"]))
max_hour_test = np.array((), dtype="int16")
for cur_id in result_df["ID"]:
    max_hour_test = np.append(arr=max_hour_test, values=np.max(X_test[X_test["ID"] == cur_id]["Hour"]))

In [37]:
# Do padding and packing for time series.
# If the patients has more records than needed, simply drop the older records.
# If the patients has less records than needed, fill in 0 so that the neural
# networks will not calculate the gradients for the 0 part.
def pad_pack(df: pd.DataFrame, max_hour: np.ndarray, seq_len: float) -> pd.DataFrame:
    result = pd.DataFrame(np.zeros((len(max_hour) * seq_len, df.shape[1])))
    pointer = 0
    i = 0
    for hour in max_hour:
        if hour < seq_len:
            # padding
            result.iloc[(i*seq_len):(i*seq_len+hour),:] = df.iloc[pointer:(pointer+hour),:]
        else:
            # packing
            result.iloc[(i*seq_len):(i*seq_len+seq_len),:] = df.iloc[(pointer+hour-seq_len):(pointer+hour),:]
        pointer += hour
        i += 1
    return result

In [38]:
# The core of the LSTM model.
# The fully connected layer will give out the numerical value of the two classes.
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, drop_out):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=drop_out)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        x = x.float()
        h0 = h0.float()
        c0 = c0.float()
        out, _ = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])
        return out

In [39]:
def training(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(train_loader):
        # Use GPU if available
        inputs, labels = inputs.to(device), labels.to(device)
        # clear the gradient
        optimizer.zero_grad()
        # Do training in batches
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    return epoch_loss

In [40]:
def evaluate(model, data_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    pred_label_arr = np.array(())
    pred_proba_arr = np.array(())
    true_label_arr = np.array(())

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(data_loader):
            # Use GPU if available
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            # Derive the prediction label
            _, pred_label = torch.max(outputs.data, 1)
            # Derive the prediction probability
            pred_proba = torch.softmax(outputs.data, 1)[:, 1]

            pred_label_arr = np.append(pred_label_arr, pred_label.detach().cpu().numpy())
            pred_proba_arr = np.append(pred_proba_arr, pred_proba.detach().cpu().numpy())
            true_label_arr = np.append(true_label_arr, labels.detach().cpu().numpy())
        
        # Calculate the validation loss, BAR, and AUC
        epoch_loss = running_loss / len(data_loader.dataset)
        epoch_bqr = balanced_accuracy_score(true_label_arr, pred_label_arr)
        epoch_auc = roc_auc_score(true_label_arr, pred_proba_arr)

    return epoch_loss, epoch_bqr, epoch_auc

In [41]:
def predict(model, data_loader, criterion, device):
    model.eval()
    pred_proba_arr = np.array(())

    with torch.no_grad():
        for i, (inputs, _) in enumerate(data_loader):
            # Use GPU if available
            inputs = inputs.to(device)

            outputs = model(inputs)

            # Derive the prediction probability
            pred_proba = torch.softmax(outputs.data, 1)[:, 1]

            pred_proba_arr = np.append(pred_proba_arr, pred_proba.detach().cpu().numpy())

    return pred_proba_arr

In [42]:
def cross_validation(
        all_x,
        all_y,
        all_test,
        max_hour,
        max_hour_test,
        num_folds,
        num_epochs,
        hidden_dim,
        layer_dim,
        batch_size,
        learning_rate,
        device,
        dropout,
        seq_len,
        random_seed
    ):
    # Model initialization
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=random_seed)
    imputer = SimpleImputer()
    scaler = StandardScaler()
    pca = PCA(random_state=random_seed)
    early_stopping_rounds = 5
    criterion = nn.CrossEntropyLoss()
    bar_list = []
    auc_list = []
    pred_df = pd.DataFrame()
    np.random.seed(seed=random_seed)
    torch.manual_seed(seed=random_seed)

    for fold, (train_index, valid_index) in enumerate(kfold.split(all_y)):
        bar = 0
        auc = 0
        print(f'Fold: {fold + 1}')
        
        train_id = all_y.loc[train_index, "ID"]
        valid_id = all_y.loc[valid_index, "ID"]
        X_train = all_x[all_x["ID"].isin(train_id)]
        X_valid = all_x[all_x["ID"].isin(valid_id)]
        y_train = all_y[all_y["ID"].isin(train_id)]["Outcome"]
        y_valid = all_y[all_y["ID"].isin(valid_id)]["Outcome"]
        X_test = all_test

        # drop ID
        X_train = X_train.drop(["ID"], axis=1)
        X_valid = X_valid.drop(["ID"], axis=1)
        X_test = X_test.drop(["ID"], axis=1)
        
        # impute
        imputer.fit(X_train)
        X_train = imputer.transform(X_train)
        X_valid = imputer.transform(X_valid)
        X_test = imputer.transform(X_test)

        # scale
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_valid = scaler.transform(X_valid)
        X_test = scaler.transform(X_test)

        # pca
        pca.fit(X_train)
        X_train = pca.transform(X_train)
        X_valid = pca.transform(X_valid)
        X_test = pca.transform(X_test)

        # padding and packing
        X_train = pd.DataFrame(X_train)
        X_valid = pd.DataFrame(X_valid)
        X_test = pd.DataFrame(X_test)
        train_max_hour = max_hour[train_index]
        valid_max_hour = max_hour[valid_index]
        X_train = pad_pack(X_train, train_max_hour, seq_len)
        X_valid = pad_pack(X_valid, valid_max_hour, seq_len)
        X_test = pad_pack(X_test, max_hour_test, seq_len)

        idx0 = np.where(y_train == 0)[0]
        idx1 = np.where(y_train == 1)[0]

        # model construction
        input_dim = X_train.shape[1]
        model = LSTMModel(input_dim, hidden_dim, layer_dim, OUTPUT_DIM, dropout).to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        
        # Convert 2D array into 3D tensor with the help of max_hour
        X_train = X_train.to_numpy()
        X_valid = X_valid.to_numpy()
        X_test = X_test.to_numpy()
        X_train = X_train.reshape((int(X_train.shape[0] / seq_len), seq_len, X_train.shape[1]))
        X_valid = X_valid.reshape((int(X_valid.shape[0] / seq_len), seq_len, X_valid.shape[1]))
        X_test = X_test.reshape((int(X_test.shape[0] / seq_len), seq_len, X_test.shape[1]))
        X_train = torch.from_numpy(X_train).to(device)
        y_train = torch.from_numpy(np.array(y_train)).to(device)
        X_valid = torch.from_numpy(X_valid).to(device)
        y_valid = torch.from_numpy(np.array(y_valid)).to(device)
        X_test = torch.from_numpy(X_test).to(device)
        temp_labels = torch.from_numpy(np.zeros_like(X_test.cpu())).to(device)
        
        # resample class 1 so that class 1 will have the same length with class 0
        if len(idx0) > (3 * len(idx1)):
            idx1 = np.random.choice(idx1, size=len(idx0))
            y_train = torch.cat((y_train[idx0], y_train[idx1]), dim=0)
            X_train = torch.cat((X_train[idx0], X_train[idx1]), dim=0)
            print("resample invoked")
        
        train = TensorDataset(X_train, y_train)
        valid = TensorDataset(X_valid, y_valid)
        test = TensorDataset(X_test, temp_labels)
        train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
        valid_loader = DataLoader(valid, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

        best_model = None
        best_val_loss = 1e10
        patience = 0
        for epoch in range(num_epochs):
            # Training
            train_loss = training(model, train_loader, criterion, optimizer, device)
            # Validating
            val_loss, val_bar, val_auc = evaluate(model, valid_loader, criterion, device)

            print(f'Epoch: {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val BAR: {val_bar:.4f}, Val AUC:{val_auc:.4f}')
            
            # Stop early if validation loss do not decrease in 5 epochs
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = model
                patience = 0
                bar = val_bar
                auc = val_auc
            else:
                patience += 1

            if patience == early_stopping_rounds:
                print('Early stopping triggered')
                break
        y_pred = predict(best_model, test_loader, criterion, device)
        pred_df[str(fold)] = y_pred
        bar_list.append(bar)
        auc_list.append(auc)
    # pred_df.to_csv("pred_prob5.csv")
    return bar_list, auc_list, pred_df

In [43]:
# The best hyperparameter set.
bar, auc, y_pred_10_model = cross_validation(
    all_x=all_x,
    all_y=all_y,
    all_test=X_test,
    max_hour=max_hour,
    max_hour_test=max_hour_test,
    num_folds=10,
    num_epochs=100,
    hidden_dim=256,
    layer_dim=2,
    batch_size=32,
    learning_rate=0.0001,
    device=DEVICE,
    dropout=0.25,
    seq_len=12,
    random_seed=20230503
)

Fold: 1
resample invoked
Epoch: 1, Train Loss: 0.5086, Val Loss: 0.3738, Val BAR: 0.8179, Val AUC:0.8773
Epoch: 2, Train Loss: 0.3409, Val Loss: 0.3309, Val BAR: 0.8197, Val AUC:0.9025
Epoch: 3, Train Loss: 0.2912, Val Loss: 0.3468, Val BAR: 0.8235, Val AUC:0.9053
Epoch: 4, Train Loss: 0.2558, Val Loss: 0.3451, Val BAR: 0.8116, Val AUC:0.8960
Epoch: 5, Train Loss: 0.2283, Val Loss: 0.3220, Val BAR: 0.8168, Val AUC:0.8996
Epoch: 6, Train Loss: 0.2028, Val Loss: 0.3072, Val BAR: 0.8154, Val AUC:0.8938
Epoch: 7, Train Loss: 0.1785, Val Loss: 0.3669, Val BAR: 0.8117, Val AUC:0.8789
Epoch: 8, Train Loss: 0.1587, Val Loss: 0.3984, Val BAR: 0.8036, Val AUC:0.8744
Epoch: 9, Train Loss: 0.1406, Val Loss: 0.3722, Val BAR: 0.8089, Val AUC:0.8753
Epoch: 10, Train Loss: 0.1217, Val Loss: 0.4363, Val BAR: 0.8072, Val AUC:0.8656
Epoch: 11, Train Loss: 0.1059, Val Loss: 0.4501, Val BAR: 0.8036, Val AUC:0.8620
Early stopping triggered
Fold: 2
resample invoked
Epoch: 1, Train Loss: 0.5123, Val Loss: 0.3

In [44]:
# 10 Fold Validation Error
print("CV BAR:", bar, "CV AUC:", auc)

CV BAR: [0.8154223729114036, 0.8102538897344833, 0.8131865400864854, 0.8460613845757361, 0.8035672659877333, 0.8262662926909751, 0.7883429187634795, 0.8415983789079897, 0.8351574319213313, 0.8351898598206133] CV AUC: [0.8937678204730459, 0.8973765432098764, 0.9032897119283122, 0.9162868049739062, 0.8982708217895561, 0.92084181194889, 0.8520129403306973, 0.8902979424601408, 0.9148307488653555, 0.9006069069230356]


In [47]:
score = y_pred_10_model.apply(lambda x: round(np.mean(x), 4), axis=1)
label = (score > 0.5).astype("int")
final_prediction = result_df.copy()
final_prediction["Outcome"] = label
final_prediction["Score"] = score
final_prediction.to_csv("test_withlabel.csv", index=False)